In [ ]:
# 구글 드라이브 마운트
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
# pip
!pip install git+https://github.com/ssut/py-hanspell.git
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

In [31]:
# 라이브러리 import
import pandas as pd
import re
import pandas as pd
from hanspell import spell_checker
from konlpy.tag import Mecab
import operator

In [44]:
df = pd.DataFrame(columns=['태그'])

def find_tag(file_path, file_name, file_format):
  # 리뷰 파일 -> 각 '브랜드-종류'별로 태그 추출하는 함수
  # 파일 로드
  column_names = 'review'
  data = pd.read_csv(file_path+'/'+file_name+file_format, delimiter = '\t', header=None, names=['review'])
  data['label'] = file_name
  # 중복 데이터 제거
  data = data.drop_duplicates(subset=['review'])
  # 결측값 제거
  data = data.dropna(subset=['review'])  # 결측값 제거
  
  # 한글만 추출(특수기호, 이모티콘, 숫자 제거)
  only_korean_list = []
  for i in data.index:
    s = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", str(data['review'][i]))
    data['review'][i] = s
  
  # 띄어쓰기, 맞춤법 교정
  for i in data.index:
    data['review'][i] = spell_checker.check(data['review'][i]).checked
  
  # 토큰화
  mecab = Mecab()
  for i in data.index:
    temp = []
    temp = mecab.pos(data['review'][i])
    temp_2 = []
    for j in range(len(temp)):
      if temp[j][1] in ['NNG', 'NNP', 'VA', 'M', 'XSN', 'XR', 'SH']:
        temp_2.append(temp[j][0])
    data['review'][i] = temp_2
  
  # 불용어 제거
  brand = file_name.split('-')[0]
  category = file_name.split('-')[1]
  stop_words = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다', '것'] + [brand, category] + ['신발', '컨버스', '좋', '잘', '신발', '같', '사이즈', '생각', '배송', '발', '구매', '적', '템', '신기', '팬츠', '발볼', '검', '흰', '감']
  for i in data.index:
    no_stop_words = []
    for j in range(len(data['review'][i])):
      if data['review'][i][j] not in stop_words:
        no_stop_words.append(data['review'][i][j])
    data['review'][i] = no_stop_words
  
  # 최대 토큰의 길이 제한
  max_token_count = 30
  index_list = []
  for i in data.index:
    if len(data['review'][i]) <= max_token_count:
      index_list.append(i)
  data = data.loc[index_list]
  
  # 단어 사전 만들기
  total_token = []
  for i in data.index:
    for j in range(len(data['review'][i])):
      total_token.append(data['review'][i][j])
  count = 0
  token_count_dictionary = dict()
  for token in set(total_token):
    count = total_token.count(token)
    token_count_dictionary[token] = count
  token_count_dictionary = sorted(token_count_dictionary.items(), key=operator.itemgetter(1), reverse=True)
  tag = []
  for i in range(10):
    tag.append(token_count_dictionary[i][0])
  tag_str = ""
  for i in range(len(tag)):
    tag_str += tag[i] + ' '
  
  return file_name, tag_str
def make_tag_df(file_name, tag_str):
  df.loc[file_name] = [tag_str]
  return df
def predict_brand_category_tag(predict_brand_category, df):
  predict_tag = df.loc[predict_brand_category].values
  return predict_tag

In [40]:
# 위에 다 import 하듯이 놓고
# 여기에만 파일 넣으면 됨~~


file_path = '/content/drive/MyDrive/GoogleColab/MiniProject/txtFile_sample'
file_name = '나이키-러닝화'
file_format = '.txt'

df = pd.DataFrame(columns=['태그'])

file_name, tag_str = find_tag(file_path, file_name, file_format)

df = make_tag_df(file_name, tag_str)

predict_brand_category_tag('나이키-러닝화', df)

array(['편하 무난 예쁘 크 감 가격 착화 디자인 이쁘 괜찮 '], dtype=object)

In [47]:
def review_training_df(file_name_list, file_path, file_format, predicted_category, ):
  df = pd.DataFrame(columns=['태그'])
  for file_name in file_name_list:
    file_name, tag_str = find_tag(file_path, file_name, file_format)
    df = make_tag_df(file_name, tag_str)
  predict_tag = predict_brand_category_tag(predicted_category, df)
  
  return predict_tag

In [49]:
file_name_list = ['나이키-Running', '반스-Canvas', '슈펜-Canvas', '아디다스-Running', '아식스-Running', '엑셀시오르-Canvas', '컨버스-Canvas', '프로스펙스-Running', '휠라-Running', 'Other-Sneakers']
file_path = '/content/drive/MyDrive/GoogleColab/MiniProject/txtFile_sample'
file_format = '.txt'
predicted_category = '나이키-Running'